In [3]:
import math
import torch
import dgl
import h5py
from HighLevelFeatures import HighLevelFeatures as HLF
from XMLHandler import XMLHandler

HLF_1_photons = HLF('photon', filename='../data/binning_dataset_1_photons.xml')
photon_file = h5py.File('../data/dataset_1_photons_1.hdf5', 'r')
xml = XMLHandler('photon', filename='../data/binning_dataset_1_photons.xml')

def get_orig_coord(bin_no, xml):
    for layer in range(len(xml.bin_edges)):
        if bin_no >= xml.bin_edges[layer]:
            continue
        else:
            layer = layer-1
            break
    bin_no = bin_no - xml.bin_edges[layer]
    r_bin = int(bin_no / xml.a_bins[layer])
    alpha_bin = bin_no % xml.a_bins[layer]
    return layer, r_bin, alpha_bin

def get_coord(bin_no, xml):
    layer, r_bin, alpha_bin = get_orig_coord(bin_no, xml)
    r = xml.r_midvalue[layer][r_bin]
    alpha = xml.alphaListPerLayer[layer][r_bin][alpha_bin]
    eta = r * math.sin(alpha)
    phi = r * math.cos(alpha)
    z = layer
    return eta, phi, z

def generate_graph_grom_incident(data, incident_no):
    bin_num = len(data[incident_no])

    edges_start = torch.arange(bin_num).repeat(bin_num)
    edges_end = torch.repeat_interleave(torch.arange(bin_num),bin_num)
    g = dgl.graph((edges_start, edges_end))

    g.ndata['x'] = torch.empty(g.num_nodes(), 3)

    for bin_no in range(len(data[incident_no])):
        eta, phi, z = get_coord(bin_no, xml)
        g.ndata['x'][bin_no] = torch.tensor([eta, phi, z])

    return g

data = photon_file["showers"][:]

graph_list = []

for incident_no in range(len(data)):
    g = generate_graph_grom_incident(data, incident_no)
    graph_list.append(g)

KeyboardInterrupt: 